In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Criando dataset de notícias
noticias = pd.DataFrame([
    {"id": 1, "titulo": "Avanços em IA", "categoria": "Inteligência Artificial e Inovação", "data": "2025-02-25", "engajamento": 120},
    {"id": 2, "titulo": "Exploração em Marte", "categoria": "Astronomia e Espaço", "data": "2025-02-20", "engajamento": 150},
    {"id": 3, "titulo": "Novo material supercondutor", "categoria": "Pesquisas Científicas", "data": "2025-02-22", "engajamento": 80},
    {"id": 4, "titulo": "Redes 6G", "categoria": "Tecnologia e Internet", "data": "2025-02-23", "engajamento": 110}
])

# Criando dataset de usuários e histórico de leitura
usuarios = pd.DataFrame([
    {"user_id": 1, "noticias_lidas": [1, 3]},
    {"user_id": 2, "noticias_lidas": [2]},
    {"user_id": 3, "noticias_lidas": []}  # Usuário novo (cold start)
])

# Modelo de recomendação baseado em conteúdo
def recomendar_noticias(user_id, top_n=3):
    usuario = usuarios[usuarios["user_id"] == user_id].iloc[0]
    noticias_lidas = usuario["noticias_lidas"]
    
    if not noticias_lidas:  # Cold start
        return noticias.sort_values(by="engajamento", ascending=False).head(top_n)
    
    # Vetorização TF-IDF das categorias
    vectorizer = TfidfVectorizer()
    matriz_tfidf = vectorizer.fit_transform(noticias["categoria"])
    
    # Similaridade entre notícias lidas e todas as outras
    sim_matrix = cosine_similarity(matriz_tfidf)
    
    # Encontrando as notícias mais similares
    indices_lidas = [noticias[noticias["id"] == n].index[0] for n in noticias_lidas]
    scores = sim_matrix[indices_lidas].mean(axis=0)
    
    # Ordenar notícias mais similares (excluindo já lidas)
    noticias["score"] = scores
    recomendadas = noticias[~noticias["id"].isin(noticias_lidas)].sort_values(by="score", ascending=False)
    
    return recomendadas.head(top_n)

# Testando recomendação
print(recomendar_noticias(1))
print(recomendar_noticias(3))


   id               titulo              categoria        data  engajamento  \
1   2  Exploração em Marte    Astronomia e Espaço  2025-02-20          150   
3   4             Redes 6G  Tecnologia e Internet  2025-02-23          110   

   score  
1    0.0  
3    0.0  
   id               titulo                           categoria        data  \
1   2  Exploração em Marte                 Astronomia e Espaço  2025-02-20   
0   1        Avanços em IA  Inteligência Artificial e Inovação  2025-02-25   
3   4             Redes 6G               Tecnologia e Internet  2025-02-23   

   engajamento  score  
1          150    0.0  
0          120    0.5  
3          110    0.0  


In [3]:
a = recomendar_noticias(1)

In [11]:
import numpy as np
import sqlite3
from lightfm import LightFM
from lightfm.data import Dataset
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 101, 75.0), (1, 102, 50.0), (1, 103, 10.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0)
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15")
}

# Criando dataset para o LightFM
dataset = Dataset()
users = set(l[0] for l in leituras)
items = set(l[1] for l in leituras)
dataset.fit(users, items)

# Criando a matriz de interações (usando porcentagem de leitura como peso)
(interactions_matrix, _, _) = dataset.build_interactions(((u, i, s / 100) for u, i, s in leituras))

# Treinando o modelo
model = LightFM(loss='warp')
model.fit(interactions_matrix, epochs=10, num_threads=2)

# Função para recomendar notícias mais recentes do melhor tema para um usuário
def recommend_news(model, dataset, user_id, noticias, n=3):
    user_internal_id = dataset.mapping()["user_id_mapping"].get(user_id)
    if user_internal_id is None:
        return "Usuário não encontrado."
    
    all_items = list(dataset.mapping()["item_id_mapping"].values())
    scores = model.predict(user_internal_id, all_items)
    ranked_items = np.argsort(-scores)
    
    item_reverse_mapping = {v: k for k, v in dataset.mapping()["item_id_mapping"].items()}
    ranked_news_ids = [item_reverse_mapping[i] for i in ranked_items]
    
    # Determinar o melhor tema com base na interação do usuário
    temas_interacoes = {}
    for noticia_id in ranked_news_ids:
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    # Escolher o tema mais relevante e recomendar as notícias mais recentes
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]

# Exemplo de recomendação para um usuário
user_id = 1
recommended_news = recommend_news(model, dataset, user_id, noticias)
print(f"Notícias recomendadas para o usuário {user_id}: {recommended_news}")


ModuleNotFoundError: No module named 'lightfm'

In [21]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

In [29]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 101, 75.0), (1, 102, 50.0), (1, 103, 10.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0)
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15")
}

# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(set(l[1] for l in leituras))
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}

# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100

# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H

# Carregar modelo salvo ou treinar um novo
try:
    with open("nmf_model.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)

# Função para recomendar notícias
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
    
    temas_interacoes = {}
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]

# Recomendação para novos usuários baseada nas notícias mais populares
def recommend_popular_news(n=3):
    popularidade = {}
    for _, noticia_id, leitura in leituras:
        if noticia_id not in popularidade:
            popularidade[noticia_id] = []
        popularidade[noticia_id].append(leitura)
    
    media_popularidade = {k: np.mean(v) for k, v in popularidade.items()}
    noticias_populares = sorted(media_popularidade.keys(), key=lambda x: (media_popularidade[x], noticias[x][1]), reverse=True)
    
    return noticias_populares[:n]

# Atualizar modelo com novas interações
def update_and_retrain(new_lectures):
    global leituras, model, W, H, matriz_interacoes
    leituras.extend(new_lectures)
    matriz_interacoes = np.zeros((num_users, num_noticias))
    for u, n, s in leituras:
        matriz_interacoes[user_map[u], noticia_map[n]] = s / 100
    model, W, H = train_model(matriz_interacoes)
    print("Modelo atualizado com novas interações e salvo.")

# Exemplo de recomendação
user_id = 1
recommended_news = recommend_news(user_id)
print(f"Notícias recomendadas para o usuário {user_id}: {recommended_news}")

# Exemplo de recomendação para novo usuário
new_user_id = 99
recommended_news_new_user = recommend_news(new_user_id)
print(f"Notícias recomendadas para o novo usuário {new_user_id}: {recommended_news_new_user}")

# Exemplo de atualização do modelo
new_data = [(1, 104, 85.0), (2, 102, 95.0)]
update_and_retrain(new_data)


Modelo carregado com sucesso.
Notícias recomendadas para o usuário 1: [104, 101]
Notícias recomendadas para o novo usuário 99: [104, 101, 102]
Modelo atualizado com novas interações e salvo.


https://chatgpt.com/share/67bf9194-65c0-8000-a1a9-33c08d728ad6

In [55]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 101, 75.0), (1, 102, 50.0), (1, 103, 10.0), (1, 106, 90.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0), (2, 107, 70.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0), (3, 108, 85.0),
    (4, 101, 50.0), (4, 106, 95.0), (4, 107, 65.0), (4, 108, 75.0),
    (5, 103, 90.0), (5, 104, 80.0), (5, 105, 70.0), (5, 109, 60.0),
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15"),
    106: ("Saúde", "2025-02-21"),
    107: ("Educação", "2025-02-19"),
    108: ("Esportes", "2025-02-23"),
    109: ("Música", "2025-02-16")
}

# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(noticias.keys())
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}

# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100

# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model2.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H

# Carregar modelo salvo ou treinar um novo
try:
    with open("nmf_model2.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)

# Função para recomendar notícias
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
    
    temas_interacoes = {}
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]

# Recomendação para novos usuários baseada nas notícias mais populares
def recommend_popular_news(n=3):
    popularidade = {}
    for _, noticia_id, leitura in leituras:
        if noticia_id not in popularidade:
            popularidade[noticia_id] = []
        popularidade[noticia_id].append(leitura)
    
    media_popularidade = {k: np.mean(v) for k, v in popularidade.items()}
    
    todas_noticias = list(noticias.keys())

    noticias_populares = sorted(
        todas_noticias, 
        key=lambda x: (media_popularidade.get(x, 0), noticias[x][1]), 
        reverse=True
    )
    
    return noticias_populares[:n]

# Atualizar modelo com novas interações
def update_and_retrain(new_lectures):
    global leituras, model, W, H, matriz_interacoes
    leituras.extend(new_lectures)
    matriz_interacoes = np.zeros((num_users, num_noticias))
    for u, n, s in leituras:
        matriz_interacoes[user_map[u], noticia_map[n]] = s / 100
    model, W, H = train_model(matriz_interacoes)
    print("Modelo atualizado com novas interações e salvo.")

# Exemplo de recomendação
for user_id in usuarios:
    recommended_news = recommend_news(user_id)
    print(f"Notícias recomendadas para o usuário {user_id}: {recommended_news}")

# Exemplo de recomendação para novo usuário
new_user_id = 99
recommended_news_new_user = recommend_news(new_user_id)
print(f"Notícias recomendadas para o novo usuário {new_user_id}: {recommended_news_new_user}")

# Exemplo de atualização do modelo
new_data = [(1, 104, 85.0), (2, 102, 95.0), (5, 107, 80.0), (4, 109, 90.0)]
update_and_retrain(new_data)


Modelo carregado com sucesso.
Notícias recomendadas para o usuário 1: [104, 101]
Notícias recomendadas para o usuário 2: [104, 101]
Notícias recomendadas para o usuário 3: [104, 101]
Notícias recomendadas para o usuário 4: [104, 101]
Notícias recomendadas para o usuário 5: [104, 101]
Notícias recomendadas para o novo usuário 99: [106, 104, 108]
Modelo atualizado com novas interações e salvo.


In [65]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 101, 75.0), (1, 102, 50.0), (1, 103, 10.0), (1, 106, 90.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0), (2, 107, 70.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0), (3, 108, 85.0),
    (4, 101, 50.0), (4, 106, 95.0), (4, 107, 65.0), (4, 108, 75.0),
    (5, 103, 90.0), (5, 104, 80.0), (5, 105, 70.0), (5, 109, 60.0)
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15"),
    106: ("Saúde", "2025-02-21"),
    107: ("Educação", "2025-02-19"),
    108: ("Esportes", "2025-02-23"),
    109: ("Música", "2025-02-16")
}

# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(noticias.keys())
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}

# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100

# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model5.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H

# Carregar modelo salvo ou treinar um novo
try:
    with open("nmf_model5.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)

# Função para adicionar um novo usuário
def add_user(user_id):
    if user_id not in user_map:
        user_map[user_id] = len(user_map)
        global num_users, matriz_interacoes, W
        num_users += 1
        matriz_interacoes = np.vstack([matriz_interacoes, np.zeros((1, num_noticias))])
        W = np.vstack([W, np.zeros((1, W.shape[1]))])
        print(f"Usuário {user_id} adicionado com sucesso.")
    else:
        print(f"Usuário {user_id} já existe.")

# Função para adicionar uma nova notícia
def add_news(noticia_id, tema, data_publicacao):
    if noticia_id not in noticias:
        noticias[noticia_id] = (tema, data_publicacao)
        global num_noticias, matriz_interacoes, H, noticias_ids
        num_noticias += 1
        matriz_interacoes = np.hstack([matriz_interacoes, np.zeros((num_users, 1))])
        H = np.hstack([H, np.zeros((H.shape[0], 1))])  # Corrigindo a expansão de H
        noticias_ids.append(noticia_id)  # Atualizando a lista de notícias
        print(f"Notícia {noticia_id} adicionada com sucesso.")
    else:
        print(f"Notícia {noticia_id} já existe.")

# Função para recomendar notícias
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
    
    temas_interacoes = {}
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]

# Função para verificar recomendação de nova notícia
def check_new_news_recommendation(user_id, noticia_id):
    if user_id not in user_map:
        print("Usuário não encontrado.")
        return
    
    user_recommendations = recommend_news(user_id, n=5)
    if noticia_id in user_recommendations:
        print(f"A nova notícia {noticia_id} foi recomendada para o usuário {user_id}!")
    else:
        print(f"A nova notícia {noticia_id} NÃO foi recomendada para o usuário {user_id}.")

# Exemplo de adição de novo usuário e recomendação
new_user_id = 99
add_user(new_user_id)
recommended_news_new_user = recommend_news(new_user_id)
print(f"Notícias recomendadas para o novo usuário {new_user_id}: {recommended_news_new_user}")

# Exemplo de adição de nova notícia
new_noticia_id = 110
add_news(new_noticia_id, "Ciência", "2025-02-24")

# Verificando se a nova notícia será recomendada
check_new_news_recommendation(1, new_noticia_id)


Modelo não encontrado. Treinando um novo...
Usuário 99 adicionado com sucesso.
Notícias recomendadas para o novo usuário 99: [104, 101]
Notícia 110 adicionada com sucesso.
A nova notícia 110 NÃO foi recomendada para o usuário 1.


In [107]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 110, 75.0), (1, 111, 95.0), (1, 103, 10.0), (1, 112, 90.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0), (2, 107, 70.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0), (3, 108, 85.0),
    (4, 101, 50.0), (4, 106, 100.0), (4, 107, 65.0), (4, 108, 75.0),
    (5, 103, 90.0), (5, 106, 100.0), (5, 105, 70.0), (5, 109, 60.0)
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15"),
    106: ("Saúde", "2025-02-21"),
    107: ("Educação", "2025-02-19"),
    108: ("Esportes", "2025-02-23"),
    109: ("Música", "2025-02-16"),
    110: ("Ciência", "2025-02-19"),
    111: ("Ciência", "2025-02-18"),
    112: ("Ciência", "2025-02-17"),
    113: ("Ciência", "2025-02-20"),
}

# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(noticias.keys())
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}

# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100

# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model10.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H

# Carregar modelo salvo ou treinar um novo
try:
    with open("nmf_model10.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)

# Função para adicionar um novo usuário
def add_user(user_id):
    if user_id not in user_map:
        user_map[user_id] = len(user_map)
        global num_users, matriz_interacoes, W
        num_users += 1
        matriz_interacoes = np.vstack([matriz_interacoes, np.zeros((1, num_noticias))])
        W = np.vstack([W, np.zeros((1, W.shape[1]))])
        print(f"Usuário {user_id} adicionado com sucesso.")
    else:
        print(f"Usuário {user_id} já existe.")

# Função para adicionar uma nova notícia
def add_news(noticia_id, tema, data_publicacao):
    if noticia_id not in noticias:
        noticias[noticia_id] = (tema, data_publicacao)
        global num_noticias, matriz_interacoes, H, noticias_ids
        num_noticias += 1
        matriz_interacoes = np.hstack([matriz_interacoes, np.zeros((num_users, 1))])
        H = np.hstack([H, np.zeros((H.shape[0], 1))])  # Corrigindo a expansão de H
        noticias_ids.append(noticia_id)  # Atualizando a lista de notícias
        print(f"Notícia {noticia_id} adicionada com sucesso.")
        # Re-treinar o modelo após adicionar a notícia
        global model, W
        model, W, H = train_model(matriz_interacoes)
    else:
        print(f"Notícia {noticia_id} já existe.")

# Função para recomendar notícias
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
    
    temas_interacoes = {}
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]

# Função para verificar recomendação de nova notícia
def check_new_news_recommendation(user_id, noticia_id):
    if user_id not in user_map:
        print("Usuário não encontrado.")
        return
    
    user_recommendations = recommend_news(user_id, n=5)
    if noticia_id in user_recommendations:
        print(f"A nova notícia {noticia_id} foi recomendada para o usuário {user_id}!")
    else:
        print(f"A nova notícia {noticia_id} NÃO foi recomendada para o usuário {user_id}.")

# Função para recomendar notícias populares
def recommend_popular_news(n=3):
    popularity = {}
    for u, noticia_id, score in leituras:
        if noticia_id not in popularity:
            popularity[noticia_id] = 0
        popularity[noticia_id] += score
    
    popular_news = sorted(popularity, key=popularity.get, reverse=True)[:n]
    return popular_news

# Exemplo de adição de novo usuário e recomendação
new_user_id = 99
add_user(new_user_id)
recommended_news_new_user = recommend_news(new_user_id)
print(f"Notícias recomendadas para o novo usuário {new_user_id}: {recommended_news_new_user}")

# Exemplo de adição de nova notícia
new_noticia_id = 114
add_news(new_noticia_id, "Ciência", "2025-02-24")

# Verificando se a nova notícia será recomendada
check_new_news_recommendation(1, new_noticia_id)

Modelo carregado com sucesso.
Usuário 99 adicionado com sucesso.


IndexError: list index out of range

In [105]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 110, 75.0), (1, 111, 95.0), (1, 103, 10.0), (1, 112, 90.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0), (2, 107, 70.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0), (3, 108, 85.0),
    (4, 101, 50.0), (4, 106, 100.0), (4, 107, 65.0), (4, 108, 75.0),
    (5, 103, 90.0), (5, 106, 100.0), (5, 105, 70.0), (5, 109, 60.0)
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15"),
    106: ("Saúde", "2025-02-21"),
    107: ("Educação", "2025-02-19"),
    108: ("Esportes", "2025-02-23"),
    109: ("Música", "2025-02-16"),
    110: ("Ciência", "2025-02-23"),
    111: ("Ciência", "2025-02-22"),
    112: ("Ciência", "2025-02-21"),
    113: ("Ciência", "2025-02-20"),
}

# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(noticias.keys())
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}

# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100

# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H

# Carregar modelo salvo ou treinar um novo
try:
    with open("nmf_model.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)

# Função para adicionar um novo usuário
def add_user(user_id):
    if user_id not in user_map:
        user_map[user_id] = len(user_map)
        global num_users, matriz_interacoes, W
        num_users += 1
        matriz_interacoes = np.vstack([matriz_interacoes, np.zeros((1, num_noticias))])
        W = np.vstack([W, np.zeros((1, W.shape[1]))])
        print(f"Usuário {user_id} adicionado com sucesso.")
    else:
        print(f"Usuário {user_id} já existe.")

# Função para adicionar uma nova notícia
def add_news(noticia_id, tema, data_publicacao):
    if noticia_id not in noticias:
        noticias[noticia_id] = (tema, data_publicacao)
        global num_noticias, matriz_interacoes, H, noticias_ids
        num_noticias += 1
        matriz_interacoes = np.hstack([matriz_interacoes, np.zeros((num_users, 1))])
        H = np.hstack([H, np.zeros((H.shape[0], 1))])
        noticias_ids.append(noticia_id)
        print(f"Notícia {noticia_id} adicionada com sucesso.")
        global model, W
        model, W, H = train_model(matriz_interacoes)
    else:
        print(f"Notícia {noticia_id} já existe.")

# Função para recomendar notícias populares
def recommend_popular_news(n=3):
    popularity = {}
    count_reads = {}
    for _, noticia_id, score in leituras:
        if noticia_id not in popularity:
            popularity[noticia_id] = 0
            count_reads[noticia_id] = 0
        popularity[noticia_id] += score
        count_reads[noticia_id] += 1
    
    weighted_popularity = {news_id: popularity[news_id] * count_reads[news_id] for news_id in popularity}
    popular_news = sorted(weighted_popularity.items(), key=lambda x: x[1], reverse=True)
    return [news_id for news_id, _ in popular_news[:n]]

# Função para recomendar notícias
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
    
    temas_interacoes = {}
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]

# Função para verificar recomendação de nova notícia
def check_new_news_recommendation(user_id, noticia_id):
    if user_id not in user_map:
        print("Usuário não encontrado.")
        return
    
    user_recommendations = recommend_news(user_id, n=5)
    print(user_recommendations)
    if noticia_id in user_recommendations:
        print(f"A nova notícia {noticia_id} foi recomendada para o usuário {user_id}!")
    else:
        print(f"A nova notícia {noticia_id} NÃO foi recomendada para o usuário {user_id}.")


# Exemplo de adição de novo usuário e recomendação
# new_user_id = 99
# add_user(new_user_id)
# recommended_news_new_user = recommend_news(new_user_id)
# print(f"Notícias recomendadas para o novo usuário {new_user_id}: {recommended_news_new_user}")

# # Exemplo de adição de nova notícia
# new_noticia_id = 114
# add_news(new_noticia_id, "Ciência", "2025-02-24")

# Verificando se a nova notícia será recomendada
check_new_news_recommendation(1, new_noticia_id)

Modelo carregado com sucesso.
[110, 111, 112, 113]
A nova notícia 114 NÃO foi recomendada para o usuário 1.


In [127]:
import numpy as np
import sqlite3
import pickle
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from datetime import datetime

# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 110, 75.0), (1, 111, 95.0), (1, 103, 10.0), (1, 112, 90.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0), (2, 107, 70.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0), (3, 108, 85.0),
    (4, 101, 50.0), (4, 106, 100.0), (4, 107, 65.0), (4, 108, 75.0),
    (5, 103, 90.0), (5, 106, 100.0), (5, 105, 70.0), (5, 109, 60.0)
]

# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15"),
    106: ("Saúde", "2025-02-21"),
    107: ("Educação", "2025-02-19"),
    108: ("Esportes", "2025-02-23"),
    109: ("Música", "2025-02-16"),
    110: ("Ciência", "2025-02-23"),
    111: ("Ciência", "2025-02-22"),
    112: ("Ciência", "2025-02-21"),
    113: ("Ciência", "2025-02-20"),
}

# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(noticias.keys())
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}

# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100

# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H

# Carregar modelo salvo ou treinar um novo
try:
    with open("nmf_model.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)

# Função para adicionar um novo usuário
def add_user(user_id):
    if user_id not in user_map:
        user_map[user_id] = len(user_map)
        global num_users, matriz_interacoes, W
        num_users += 1
        matriz_interacoes = np.vstack([matriz_interacoes, np.zeros((1, num_noticias))])
        W = np.vstack([W, np.zeros((1, W.shape[1]))])
        print(f"Usuário {user_id} adicionado com sucesso.")
    else:
        print(f"Usuário {user_id} já existe.")

# Função para adicionar uma nova notícia
def add_news(noticia_id, tema, data_publicacao):
    if noticia_id not in noticias:
        noticias[noticia_id] = (tema, data_publicacao)
        global num_noticias, matriz_interacoes, H, noticias_ids
        num_noticias += 1
        matriz_interacoes = np.hstack([matriz_interacoes, np.zeros((num_users, 1))])
        H = np.hstack([H, np.zeros((H.shape[0], 1))])
        noticias_ids.append(noticia_id)
        print(f"Notícia {noticia_id} adicionada com sucesso.")
        global model, W
        model, W, H = train_model(matriz_interacoes)
    else:
        print(f"Notícia {noticia_id} já existe.")

# Função para recomendar notícias populares
def recommend_popular_news(n=3):
    popularity = {}
    count_reads = {}
    for user_id, noticia_id, score in leituras:
        if score > 50.0:
            continue
        if noticia_id not in popularity:
            popularity[noticia_id] = 0
            count_reads[noticia_id] = 0
        popularity[noticia_id] += score
        count_reads[noticia_id] += 1
    
    weighted_popularity = {news_id: popularity[news_id] * count_reads[news_id] for news_id in popularity}
    popular_news = sorted(weighted_popularity.items(), key=lambda x: x[1], reverse=True)
    return [news_id for news_id, _ in popular_news[:n]]

# Função para recomendar notícias
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
    
    recomendadas = []
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        if any(u == user_id and n == noticia_id and s > 50.0 for u, n, s in leituras):
            continue
        recomendadas.append(noticia_id)
        if len(recomendadas) >= n:
            break
    
    return recomendadas


# Exemplo de adição de novo usuário e recomendação
# new_user_id = 99
# add_user(new_user_id)
# recommended_news_new_user = recommend_news(new_user_id)
# print(f"Notícias recomendadas para o novo usuário {new_user_id}: {recommended_news_new_user}")

# # Exemplo de adição de nova notícia
new_noticia_id = 114
add_news(new_noticia_id, "Ciência", "2025-02-24")

# Verificando se a nova notícia será recomendada
check_new_news_recommendation(1, new_noticia_id)

Modelo carregado com sucesso.
Notícia 114 adicionada com sucesso.
A nova notícia 114 NÃO foi recomendada para o usuário 1.


In [129]:
user_id=1
user_idx = user_map[user_id]
scores = np.dot(W[user_idx], H)
ranked_news_idx = np.argsort(scores)[::-1]
recomendadas = []
for idx in ranked_news_idx:
    noticia_id = noticias_ids[idx]
    if any(u == user_id and n == noticia_id and s > 50.0 for u, n, s in leituras):
        continue
    recomendadas.append(noticia_id)
    if len(recomendadas) >= n:
        break

In [131]:
noticias_ids

[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]

In [133]:
ranked_news_idx

array([10, 11,  9,  2,  7,  1,  4,  3, 13, 12,  8,  6,  5,  0],
      dtype=int64)

In [135]:
recomendadas

[103, 108, 102, 105, 104, 114, 113, 109, 107, 106, 101]

In [137]:
leituras

[(1, 110, 75.0),
 (1, 111, 95.0),
 (1, 103, 10.0),
 (1, 112, 90.0),
 (2, 101, 80.0),
 (2, 104, 90.0),
 (2, 105, 60.0),
 (2, 107, 70.0),
 (3, 102, 70.0),
 (3, 103, 40.0),
 (3, 105, 20.0),
 (3, 108, 85.0),
 (4, 101, 50.0),
 (4, 106, 100.0),
 (4, 107, 65.0),
 (4, 108, 75.0),
 (5, 103, 90.0),
 (5, 106, 100.0),
 (5, 105, 70.0),
 (5, 109, 60.0)]

📌 Objetivo do Código
Este código implementa um sistema de recomendação de notícias utilizando Filtragem Colaborativa com o modelo NMF (Non-Negative Matrix Factorization).

Ele recomenda notícias para os usuários com base nas suas interações anteriores (percentual de leitura de cada notícia). Para novos usuários (cold start), ele recomenda as notícias mais populares entre os usuários existentes.

🔹 1. Simulação de Dados
python
Copiar
Editar
# Simulação de dados de interações (usuário, notícia, porcentagem de leitura)
leituras = [
    (1, 101, 75.0), (1, 102, 50.0), (1, 103, 10.0),
    (2, 101, 80.0), (2, 104, 90.0), (2, 105, 60.0),
    (3, 102, 70.0), (3, 103, 40.0), (3, 105, 20.0)
]
💡 O que são essas interações?

Cada tupla representa um usuário que leu uma notícia e qual porcentagem dessa notícia ele consumiu.
Por exemplo, (1, 101, 75.0) significa que o usuário 1 leu 75% da notícia 101.
python
Copiar
Editar
# Simulação de dados de notícias (id, tema, data de publicação)
noticias = {
    101: ("Crime e Justiça", "2025-02-20"),
    102: ("Economia", "2025-02-18"),
    103: ("Política", "2025-02-17"),
    104: ("Crime e Justiça", "2025-02-22"),
    105: ("Tecnologia", "2025-02-15")
}
💡 O que significa isso?

Cada notícia tem um ID, um tema e uma data de publicação.
Por exemplo, 101: ("Crime e Justiça", "2025-02-20") significa que a notícia 101 fala sobre Crime e Justiça e foi publicada em 20 de fevereiro de 2025.
🔹 2. Criando a Matriz de Interação
Antes de treinar o modelo, transformamos os dados em uma matriz de interações.

python
Copiar
Editar
# Criando mapeamento de índices
usuarios = list(set(l[0] for l in leituras))
noticias_ids = list(set(l[1] for l in leituras))
user_map = {u: i for i, u in enumerate(usuarios)}
noticia_map = {n: i for i, n in enumerate(noticias_ids)}
💡 Por que precisamos disso?

Os modelos trabalham melhor com índices numéricos sequenciais ao invés de identificadores arbitrários.
user_map e noticia_map transformam IDs de usuários e notícias em índices numéricos.
python
Copiar
Editar
# Criando a matriz de interações
num_users = len(usuarios)
num_noticias = len(noticias_ids)
matriz_interacoes = np.zeros((num_users, num_noticias))
for u, n, s in leituras:
    matriz_interacoes[user_map[u], noticia_map[n]] = s / 100
💡 O que isso faz?

Criamos uma matriz num_users × num_noticias com zeros.
Para cada interação (usuário, notícia, percentual de leitura), preenchemos a matriz com valores entre 0 e 1.
Exemplo: Se um usuário leu 75% de uma notícia, armazenamos 0.75 na posição correspondente.
🔹 3. Treinando o Modelo NMF
Agora treinamos o modelo NMF para encontrar padrões nos dados.

python
Copiar
Editar
# Treinando o modelo NMF
def train_model(matriz_interacoes):
    model = NMF(n_components=5, init='random', random_state=42)
    W = model.fit_transform(matriz_interacoes)
    H = model.components_
    
    with open("nmf_model.pkl", "wb") as f:
        pickle.dump((model, W, H), f)
    return model, W, H
💡 O que é o NMF?

O NMF (Non-Negative Matrix Factorization) decompõe a matriz em dois conjuntos de fatores:
W representa os perfis dos usuários.
H representa os perfis das notícias.
O modelo é salvo no arquivo "nmf_model.pkl" para evitar treinar toda vez.
🔹 4. Carregar Modelo ou Treinar Novamente
python
Copiar
Editar
try:
    with open("nmf_model.pkl", "rb") as f:
        model, W, H = pickle.load(f)
    print("Modelo carregado com sucesso.")
except FileNotFoundError:
    print("Modelo não encontrado. Treinando um novo...")
    model, W, H = train_model(matriz_interacoes)
💡 Por que carregar o modelo?

Para evitar retrainamento desnecessário e economizar tempo.
🔹 5. Recomendação de Notícias
Agora usamos o modelo para recomendar notícias.

python
Copiar
Editar
def recommend_news(user_id, n=3):
    if user_id not in user_map:
        return recommend_popular_news(n)
    
    user_idx = user_map[user_id]
    scores = np.dot(W[user_idx], H)
    ranked_news_idx = np.argsort(scores)[::-1]
💡 Como funciona?

Se o usuário for novo, ele recebe notícias populares.
Se ele já interagiu antes, calculamos um score de preferência multiplicando W[user] × H.
As notícias são ordenadas por esse score.
python
Copiar
Editar
    temas_interacoes = {}
    for idx in ranked_news_idx:
        noticia_id = noticias_ids[idx]
        tema, data_pub = noticias[noticia_id]
        if tema not in temas_interacoes:
            temas_interacoes[tema] = []
        temas_interacoes[tema].append((noticia_id, datetime.strptime(data_pub, "%Y-%m-%d")))
    
    melhor_tema = max(temas_interacoes, key=lambda t: len(temas_interacoes[t]))
    noticias_relevantes = sorted(temas_interacoes[melhor_tema], key=lambda x: x[1], reverse=True)[:n]
    
    return [noticia_id for noticia_id, _ in noticias_relevantes]
💡 Como garantimos que as notícias são recentes?

Agrupamos notícias por tema.
Escolhemos o tema mais relevante baseado na quantidade de interações.
Ordenamos as notícias desse tema pela data de publicação.
🔹 6. Recomendação para Novos Usuários
python
Copiar
Editar
def recommend_popular_news(n=3):
    popularidade = {}
    for _, noticia_id, leitura in leituras:
        if noticia_id not in popularidade:
            popularidade[noticia_id] = []
        popularidade[noticia_id].append(leitura)
    
    media_popularidade = {k: np.mean(v) for k, v in popularidade.items()}
    noticias_populares = sorted(media_popularidade.keys(), key=lambda x: (media_popularidade[x], noticias[x][1]), reverse=True)
    
    return noticias_populares[:n]
💡 Como funciona?

Calculamos a média de leitura de cada notícia.
Ordenamos as notícias pela popularidade e recência.
🔹 7. Atualizando e Retreinando o Modelo
python
Copiar
Editar
def update_and_retrain(new_lectures):
    global leituras, model, W, H, matriz_interacoes
    leituras.extend(new_lectures)
    matriz_interacoes = np.zeros((num_users, num_noticias))
    for u, n, s in leituras:
        matriz_interacoes[user_map[u], noticia_map[n]] = s / 100
    model, W, H = train_model(matriz_interacoes)
    print("Modelo atualizado com novas interações e salvo.")
💡 Por que isso é importante?

Sempre que novos dados são adicionados, o modelo precisa ser atualizado para aprender novas preferências.
Isso cobre tudo! Se tiver dúvidas ou quiser modificar algo, me avise! 🚀
